# Manual Testing Notebook

This notebook allows you to step through the logic of the Credit Risk Reporting application manually.

In [ ]:
import os
import sys
import yaml
import pandas as pd

# Ensure we can import from src
sys.path.append(os.path.abspath('..'))

from src.data_generator import SyntheticDataLoader
from src.calculations import CreditRiskFormulas
from src.ai_engine import RiskReporter

## 1. Load Configuration

In [ ]:
with open("../config.yaml", "r") as f:
    config = yaml.safe_load(f)

print("Alpha:", config['thresholds']['significance_level_alpha'])
print("Model:", config['ai_settings']['model'])

## 2. Generate Synthetic Data

In [ ]:
loader = SyntheticDataLoader()

df_auc = loader.generate_auc_data()
print("AUC Data Head:")
display(df_auc.head())

In [ ]:
df_calib = loader.generate_calibration_data()
print("Calibration Data:")
display(df_calib)

In [ ]:
df_scores = loader.generate_score_data()
print("Score Data Head:")
display(df_scores.head())

## 3. Calculate Statistics

In [ ]:
calculator = CreditRiskFormulas(config)

auc_stats = calculator.evaluate_auc(df_auc)
print("AUC Stats:", auc_stats)

chi_stats = calculator.calculate_chi_square(df_calib)
print("Chi-Square Stats:", chi_stats)

ttest_stats = calculator.calculate_ttest(df_scores)
print("T-Test Stats:", ttest_stats)

In [ ]:
binomial_results = calculator.calculate_binomial_test(df_calib)
print("Binomial Test Results:")
display(binomial_results)

In [ ]:
jeffrey_results = calculator.calculate_jeffrey_test(df_calib)
print("Jeffrey Test Results:")
display(jeffrey_results)

## 4. AI Reporting (Optional)

In [ ]:
# To run this, ensure you have a .env file with OPENAI_API_KEY in the root directory
run_ai = False

if run_ai:
    reporter = RiskReporter("../config.yaml")
    report = reporter.generate_report(
        auc_stats, chi_stats, binomial_results, jeffrey_results, ttest_stats
    )
    print(report)
else:
    print("Skipping AI generation (set run_ai=True to run)")